In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from sklearn.metrics import precision_score, recall_score, accuracy_score, mean_squared_error
from PIL import Image

# Custom Dataset for WikiArt
class WikiArtDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.files = []

        for label, cls in enumerate(self.classes):
            class_dir = os.path.join(root_dir, cls)
            for file_name in os.listdir(class_dir):
                file_path = os.path.join(class_dir, file_name)
                if os.path.isfile(file_path):
                    self.files.append((file_path, label))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path, label = self.files[idx]
        try:
            image = Image.open(file_path).convert("RGB")
        except Exception as e:
            raise RuntimeError(f"Error loading image {file_path}: {e}")
        if self.transform:
            image = self.transform(image)
        return image, label

# Define a Classification Model with Softmax
class SoftmaxClassificationModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SoftmaxClassificationModel, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = self.fc(x)
        return nn.Softmax(dim=1)(x)  # Apply softmax to output

if __name__ == "__main__":
    data_path = r"C:\Users\yozev\OneDrive\Desktop\artFiltered"  # Path to filtered dataset

    # Define image transformations
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    # Load dataset
    dataset = WikiArtDataset(root_dir=data_path, transform=transform)

    # Split into train and test datasets
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    print(f"Number of training samples: {len(train_dataset)}")
    print(f"Number of testing samples: {len(test_dataset)}")

    # Initialize the model
    input_size = 256 * 256 * 3
    num_classes = len(dataset.classes)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = SoftmaxClassificationModel(input_size, num_classes).to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    # Training loop
    epochs = 50

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for images, labels in train_loader:
            images = images.view(images.size(0), -1).to(device)  # Flatten images
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")

    # Evaluation
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(images.size(0), -1).to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)  # Get predicted class with highest probability

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average="weighted")
    recall = recall_score(all_labels, all_preds, average="weighted")

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

Number of training samples: 51800
Number of testing samples: 12951
Epoch [1/50], Loss: 2.7970
Epoch [2/50], Loss: 2.7965
Epoch [3/50], Loss: 2.7965
Epoch [4/50], Loss: 2.7965



KeyboardInterrupt

